In [1]:
import numpy as np
import cv2
import pickle
import glob, os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.ndimage.measurements import label

In [3]:
# Load model parameters
if not os.path.exists("./model_pickle.p"):
    print("Train the classifier first!")
else:    
    model_pickle = pickle.load( open( "./model_pickle.p", "rb" ))
    svc = model_pickle['model']
    feature_params = model_pickle['params']
    print("Classifier loaded.")
    del model_pickle
    
# load the feature pickle
if not os.path.exists("./feature_pickle.p"):
    print("Save feature parameters first!")
else:
    feature_pickle = pickle.load(open("./feature_pickle.p", 'rb'))
    X_scaler = feature_pickle['X_scaler']
    print("Feature parameters loaded.")
    del feature_pickle

Classifier loaded.
Feature parameters loaded.


In [4]:
from VehicleDetection import VehicleDetection

VehicleDetection = VehicleDetection(feature_params)

In [5]:
def pipeline(img):
    scales = [1, 1.3, 1.5]
    out_img, box_lists = VehicleDetection.multi_scale(img, scales, svc, X_scaler)
    draw_img = VehicleDetection.draw_heat_box(img, box_lists, 4)
    
    out_img = cv2.resize(out_img, (320,240))
    
    output_image = np.ones((img.shape[0], img.shape[1], 3))*255
    output_image[0:img.shape[0], 0:img.shape[1]] = draw_img
    
    output_image[8:252, 8:332] = np.ones((244, 324, 3))*255
    output_image[10:250, 10:330] = out_img
    
    font_type = cv2.FONT_HERSHEY_DUPLEX 
    cv2.putText(output_image, "Detected boxes", (30, 30), font_type, 0.8, (255,255,255), 2)
#     cv2.putText(output_image, "Filtered boxes", (20, 640), font_type, 0.8, (255,255,255), 2)

    return output_image

In [6]:
from moviepy.editor import VideoFileClip
video_output = '../test_video_output.mp4'
clip1 = VideoFileClip("../test_video.mp4");
white_clip = clip1.fl_image(pipeline) 
%time white_clip.write_videofile(video_output, audio = False, threads=4);

[MoviePy] >>>> Building video ../test_video_output.mp4
[MoviePy] Writing video ../test_video_output.mp4


 97%|████████████████████████████████████████▉ | 38/39 [01:21<00:02,  2.18s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../test_video_output.mp4 

Wall time: 1min 23s


In [7]:
from IPython.display import HTML
HTML("""
<video width="800" height="600" controls>
  <source src="{0}">
</video>
""".format(video_output))

In [8]:
from moviepy.editor import VideoFileClip
video_output2 = '../project_video_output.mp4'
clip2 = VideoFileClip("../project_video.mp4");
white_clip = clip2.fl_image(pipeline) 
%time white_clip.write_videofile(video_output2, audio = False, threads=4);

[MoviePy] >>>> Building video ../project_video_output.mp4
[MoviePy] Writing video ../project_video_output.mp4


100%|█████████████████████████████████████▉| 1260/1261 [44:53<00:02,  2.10s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../project_video_output.mp4 

Wall time: 44min 55s


In [9]:
from IPython.display import HTML
HTML("""
<video width="800" height="600" controls>
  <source src="{0}">
</video>
""".format(video_output2))